In [4]:
from methods.unet import UNet_encoder
import torch
from utils_loader import test_idea_dataloader_e2
config = {
    'batch_size': 128,
    'epochs':200,
    'fsl': False,
    'model': 'unet',
    'seed': 73054772,
    'dataset': 'physiq'
}
train_loader, val_loader, test_loader = test_idea_dataloader_e2(config)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet_encoder(in_channels=6, out_channels=5, cnn_embed_dims=[64, 128, 356]).float().to(device)
model.load_state_dict(torch.load('saved_model/opportunity_unet_test_EXP2_B.pth', map_location=device))

<All keys matched successfully>

In [16]:
from captum.attr import LayerGradCam, LayerAttribution
# print(model.backbone[-2].conv2)
layer_gc = LayerGradCam(model, model.backbone[-2].conv2)
# get one sample from 0 and 1 label:
inputs, labels = next(iter(test_loader))
input_0 = inputs[torch.where(labels == 0)[0]][0].unsqueeze(0).float()
input_1 = inputs[torch.where(labels == 1)[0]][0].unsqueeze(0).float()
label_0 = labels[torch.where(labels == 0)[0]][0].unsqueeze(0).float()
label_1 = labels[torch.where(labels == 1)[0]][0].unsqueeze(0).float()
print(input_0.shape)
attr = layer_gc.attribute(input_0, label_0)

# upsampled_attr = LayerAttribution.interpolate(attr, (32, 32))

torch.Size([1, 50, 6])


AssertionError: Tensor target dimension torch.Size([1]) is not valid. torch.Size([5])